# Init

## connect db

In [5]:
import sqlite3
con = sqlite3.connect('../../pickles/swarog_data/swarog.sqlite')

In [213]:
con.commit()

In [214]:
con.close()

In [2]:
!git clone git@github.com:DrPenknife/pybars.git

fatal: destination path 'pybars' already exists and is not an empty directory.


In [4]:
!ssh-keygen -f "/home/rkozik/.ssh/known_hosts" -R "github.com"

# Host github.com found: line 1
# Host github.com found: line 5
/home/rkozik/.ssh/known_hosts updated.
Original contents retained as /home/rkozik/.ssh/known_hosts.old


In [9]:
!cd pybars && pwd

/media/rkozik/02FF-A831/repos/swarog_exp/experiments/fngraphs/pybars


In [10]:
!cd pybars && git pull

Already up to date.


In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import pybars.charts as mygraph
mygraph.bindSql(con)

binding sql None


## show tables

In [62]:
%%sql show=df db=../../pickles/swarog_data/swarog.sqlite
select * from sqlite_master

using ../../pickles/swarog_data/swarog.sqlite


,type,name,tbl_name,rootpage,sql
0,table,raw,raw,2,"CREATE TABLE raw\n (dataset TEXT, ..."
1,table,tfidf,tfidf,353265,"CREATE TABLE tfidf(gid INT,dataset TEXT,id INT)"
2,table,bertcls,bertcls,3,CREATE TABLE bertcls\n (dataset TE...
3,table,rawsearch,rawsearch,0,CREATE VIRTUAL TABLE rawsearch USING fts5(data...
4,table,rawsearch_data,rawsearch_data,185159,CREATE TABLE 'rawsearch_data'(id INTEGER PRIMA...
5,table,rawsearch_idx,rawsearch_idx,185298,"CREATE TABLE 'rawsearch_idx'(segid, term, pgno..."
6,table,rawsearch_content,rawsearch_content,185299,CREATE TABLE 'rawsearch_content'(id INTEGER PR...
7,table,rawsearch_docsize,rawsearch_docsize,185300,CREATE TABLE 'rawsearch_docsize'(id INTEGER PR...
8,table,rawsearch_config,rawsearch_config,185301,"CREATE TABLE 'rawsearch_config'(k PRIMARY KEY,..."
9,table,covid_fake_news_search,covid_fake_news_search,0,CREATE VIRTUAL TABLE covid_fake_news_search us...


In [18]:
%%sql show=raw
select * from covid_fake_news_search
limit 5 offset 10


[(1, 'Good oral hygiene destroys the coronavirus and prevents its spread'),
 (1,
  'A video shows Pakistani minister of Health slapping those who do not obey official measures'),
 (1,
  'A Brazilian deputee says the pandemic has started to decrease in his state, Rio Grande do Sul'),
 (1, 'WHO warns against consuming cabbage to prevent COVID-19'),
 (1,
  'A picture shows hydroxychloroquine is being sold on public trains in Rio de Janeiro')]

# Create extra tabels

Tworze tabele dla datasetow covidowych zeby zbadac grafy na czyms prostym

In [ ]:
%%sql show=raw
create virtual table covid_fake_news_search using fts5(label, body)

'(empty set)'

In [90]:
%%sql show=raw
insert into covid_fake_news_search select label, body from raw where dataset in ("covid_fake_news")

'(empty set)'

# NER

In [25]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")


In [55]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = 'Germany representative to the European Union veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer.'
example = 'Werner said consumers should buy sheepmeat from countries other than Britain'
example = 'WHO suggests oral hygiene to prevent the spread of coronavirus.'
ner_results = nlp(example)
for ner in ner_results:
    print(ner)

{'entity': 'B-ORG', 'score': 0.9988845, 'index': 1, 'word': 'WHO', 'start': 0, 'end': 3}


In [56]:
import pandas as pd
pd.DataFrame(ner_results)

,entity,score,index,word,start,end
0,B-ORG,0.998884,1,WHO,0,3


# T5

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

# Tokenize TFIDF


In [46]:
%%sql show=df
select * from raw where dataset in ("covid_fake_news") limit 5 offset 10

,dataset,id,body,label
0,covid_fake_news,10,Good oral hygiene destroys the coronavirus and...,1
1,covid_fake_news,11,A video shows Pakistani minister of Health sla...,1
2,covid_fake_news,12,A Brazilian deputee says the pandemic has star...,1
3,covid_fake_news,13,WHO warns against consuming cabbage to prevent...,1
4,covid_fake_news,14,A picture shows hydroxychloroquine is being so...,1


In [47]:
import pandas as pd
df=pd.read_sql_query("""
select body, label from covid_fake_news_search where 1 
-- limit 5 offset 10
""",con)

In [138]:
df.shape

(8972, 2)

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords

# Download stopwords list
nltk.download('wordnet')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

# Interface lemma tokenizer from nltk with sklearn
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`', "'"]
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

# Lemmatize the stop words
tokenizer=LemmaTokenizer()
token_stop = tokenizer(' '.join(stop_words))


# Create TF-idf model
tfidf_vectorizer = TfidfVectorizer(stop_words=token_stop)

doc_vectors = tfidf_vectorizer.fit_transform(df['body'])

 

[nltk_data] Downloading package wordnet to /home/rkozik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/rkozik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## search related document 

In [49]:
import numpy as np
import re

vocabulary_tfidf_words = tfidf_vectorizer.get_feature_names_out()

def tfidf_to_words(vec):
    li=sorted(list(zip(vec[np.where(vec > 0)], vocabulary_tfidf_words[np.where(vec > 0)[0]])), key=lambda tup: -tup[0])
    # returns array[(tf,word),...]
    return li

def get_related_docs(_sentence, max_keywords=7):
    # get tf-idf values
    vec = tfidf_vectorizer.transform([_sentence]).toarray()[0]
    
    # zip (word, word_wieght)
    # get sorted by TF-IDF score
    _words = tfidf_to_words(vec)

    conn = sqlite3.connect('../../pickles/swarog_data/swarog.sqlite')
    _resp=[]
    hitkeywords = {}
    # in which doc is the word_id (iterate over)
    for _range in range(0,min(max_keywords,len(_words))):
        x=_words[_range]

        #replace non-ascii to spapces
        _chain = (x[0],re.sub(r'[^a-zA-Z0-9]', ' ', x[1]).split(" ")[0])

        if len(_chain[1]) == 0:
            continue
        
        c = conn.cursor()
        c.execute(f"""select rowid from covid_fake_news_search where body match '{_chain[1]}' """)
        docsids = c.fetchall()
        r=[x[0] for x in docsids] # ids of docs containing word 
        _resp.extend(r)
        for x in docsids:
            # document_id -> [array of keywords] map
            if not x[0] in hitkeywords:
                hitkeywords[x[0]] = []

            hitkeywords[x[0]].append(_chain[1])

 
    from collections import Counter
    ctr=Counter(_resp)
    common = ctr.most_common(10) # 10 top hits

    #print("common",common)
    resp = []
    # collect the top 10 documents
    for _docid in common:
        c.execute(f"""select label, "covid_fake_news" as dataset, body from covid_fake_news_search where rowid = {_docid[0]} """)
        _results_hits = c.fetchall()[0]
        _doc_tfidf_vec = tfidf_vectorizer.transform([_results_hits[2]]).toarray()[0]
        _doc_words = [x[1] for x in tfidf_to_words(_doc_tfidf_vec)]
        resp.append({
            #'text':_results_hits[2],
            'rowid': _docid[0],
            'length': len(_results_hits[2]),
            'hit_count': len(hitkeywords[_docid[0]]),
            'label':_results_hits[0],
            'dataset':_results_hits[1],
            'hit_keywords': hitkeywords[_docid[0]],
            'keywords': _doc_words[:2*max_keywords],
            'distance':1.0 - _docid[1]*1.0/min(max_keywords,len(_words))})
    conn.close()
    
    #print(resp)
    return resp,_words[:2*max_keywords]
    #_resp

    

In [19]:
doc_vectors.shape

(8972, 12662)

### example

In [105]:
resp,words=get_related_docs("Good oral hygiene destroys the coronavirus and prevents its spread.")
pd.DataFrame(resp)


common [(11, 7), (8839, 2), (1410, 2), (4655, 2), (391, 2), (3079, 2), (3128, 2), (7891, 2), (4830, 2), (5567, 2)]


,rowid,length,hit_count,label,dataset,hit_keywords,keywords,distance
0,11,66,7,1,covid_fake_news,"[oral, hygiene, destroys, good, prevents, its,...","[oral, hygiene, destroys, good, prevents, its,...",0.000000
1,8839,133,2,1,covid_fake_news,"[hygiene, its]","[wholesale, exposing, hygiene, environment, lo...",0.714286
2,1410,183,2,0,covid_fake_news,"[good, spread]","[likely, less, alleviate, pretty, concerns, ki...",0.714286
3,4655,249,2,1,covid_fake_news,"[good, spread]","[establishments, œtotal, curb, announce, items...",0.714286
4,391,93,2,1,covid_fake_news,"[prevents, spread]","[methanol, covid19, videos, prevents, saying, ...",0.714286
5,3079,51,2,1,covid_fake_news,"[prevents, spread]","[sea, prevents, drinking, water, spread, 19, c...",0.714286
6,3128,45,2,1,covid_fake_news,"[prevents, spread]","[aromatherapy, prevents, spread, 19, covid]",0.714286
7,7891,279,2,1,covid_fake_news,"[prevents, its]","[air, muzzles, pharmacists, settle, diameter, ...",0.714286
8,4830,229,2,1,covid_fake_news,"[its, spread]","[œlowâ, œricherâ, relatively, doh, francisco, ...",0.714286
9,5567,83,2,1,covid_fake_news,"[its, spread]","[repentance, sinsâ, œadmits, declares, america...",0.714286


# Create graph

In [40]:
!rm '../../pickles/swarog_data/graph.sqlite'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
import sqlite3
graph = sqlite3.connect('../../pickles/swarog_data/graph.sqlite')

In [51]:
graph.commit()

In [52]:
graph.close()

In [42]:
graph.execute("""
CREATE TABLE IF NOT EXISTS nodes (
    body TEXT,
    type TEXT,
    id   TEXT GENERATED ALWAYS AS (json_extract(body, '$.id')) VIRTUAL NOT NULL UNIQUE
);
""")

graph.execute("""
CREATE INDEX IF NOT EXISTS id_idx ON nodes(id);
""")

graph.execute("""
CREATE TABLE IF NOT EXISTS edges (
    source     TEXT,
    target     TEXT,
    properties TEXT,
    UNIQUE(source, target, properties) ON CONFLICT REPLACE,
    FOREIGN KEY(source) REFERENCES nodes(id),
    FOREIGN KEY(target) REFERENCES nodes(id)
);
""")

graph.execute("""
CREATE INDEX IF NOT EXISTS source_idx ON edges(source);
""")

graph.execute("""
CREATE INDEX IF NOT EXISTS target_idx ON edges(target);
""")


## Extract nodes

In [43]:
import json 
for row in con.execute("select rowid,body,label from covid_fake_news_search"):
    docid, body, label = row
    j = json.dumps({'id':docid, 'body': body, 'label':label}, default=str)
    graph.execute("insert OR ignore into nodes(body,type) values(?,'document')" , [j])
graph.commit()

## Edges

In [50]:
for row in con.execute("select rowid,body from covid_fake_news_search"):
    docid, body = row
    resp,words=get_related_docs(body)
    for elem in resp:
        j = json.dumps(elem, default=str)
        graph.execute("insert OR ignore into edges(source, target, properties) values(?,?,?)" , [docid,elem['rowid'],j])
    #print(resp)
graph.commit()

In [71]:
%%sql show=df db=../../pickles/swarog_data/graph.sqlite
select * from sqlite_master where type="table"

using ../../pickles/swarog_data/graph.sqlite


,type,name,tbl_name,rootpage,sql
0,table,nodes,nodes,2,"CREATE TABLE nodes (\n body TEXT,\n id ..."
1,table,edges,edges,5,"CREATE TABLE edges (\n source TEXT,\n ..."


In [135]:
%%sql show=df db=../../pickles/swarog_data/graph.sqlite
select * from nodes limit 10

using ../../pickles/swarog_data/graph.sqlite


,body,id
0,"{""id"": 1, ""body"": ""A post claims compulsory va...",1
1,"{""id"": 2, ""body"": ""A photo claims that this pe...",2
2,"{""id"": 3, ""body"": ""Post about a video claims t...",3
3,"{""id"": 4, ""body"": ""All deaths by respiratory f...",4
4,"{""id"": 5, ""body"": ""The dean of the College of ...",5
5,"{""id"": 6, ""body"": ""Households with COVID-19 pa...",6
6,"{""id"": 7, ""body"": ""A chain lists recommendatio...",7
7,"{""id"": 8, ""body"": ""60,000 Argentinian companie...",8
8,"{""id"": 9, ""body"": ""Social media posts criticiz...",9
9,"{""id"": 10, ""body"": ""CDC has released an update...",10


In [94]:
%%sql show=df db=../../pickles/swarog_data/graph.sqlite
select 
    source,target,
    json_extract(properties, '$.hit_count') as hits , 
    json_extract(properties, '$.distance') as distance 
from edges 
where 1
and source<>target 
and distance < 0.6 
order by distance 
asc limit 10

using ../../pickles/swarog_data/graph.sqlite


,source,target,hits,distance
0,17,7891,6,0.0
1,25,658,6,0.0
2,29,53,7,0.0
3,50,52,7,0.0
4,52,50,7,0.0
5,53,29,7,0.0
6,64,793,7,0.0
7,118,165,3,0.0
8,165,118,3,0.0
9,243,251,7,0.0


In [1]:
%%sql show=df db=../../pickles/swarog_data/graph.sqlite
select 
    source,count(*)
from edges 
group by 1
order by 2 desc
limit 10

UsageError: Cell magic `%%sql` not found.


In [ ]:
for row in con.execute("select rowid,body from covid_fake_news_search limit 1 offset 7890"):
    docid, body = row
    print("Q:",docid, body)
    resp,words=get_related_docs(body)
    print("words:",words)
    for r in resp:
        print("  ->",r)

In [ ]:
for row in con.execute("select rowid,body from covid_fake_news_search limit 1 offset 250 "):
    docid, body = row
    print("Q:",docid, body)
    resp,words=get_related_docs(body)
    print("words:",words)
    for r in resp:
        print("  ->",r)

In [125]:
[ {"name":id[0]} for id in graph.execute("select * from  limit 5").fetchall() ]

[{'name': '1'}, {'name': '2'}, {'name': '3'}, {'name': '4'}, {'name': '5'}]

In [23]:
import pybars.network as network
nodes = [ {"name":id[0]} for id in graph.execute("select   json_extract(body, '$.body')   from nodes limit 1500").fetchall() ]
links = [
    {"source": row[0], "target": row[1], "category":0} for row in graph.execute(
    """
        select 
        source-1,target-1,
        json_extract(properties, '$.hit_count') as hits , 
        json_extract(properties, '$.distance') as distance 
        from edges 
        where 1
        and source<>target 
        and distance < 0.6 
        order by distance 
        asc
    """).fetchall()
]
categories = [{"name":"Man"},{"name":"Woman"}]

network.draw({
    "graph":{
        "nodes":nodes,
        "links":links,
        "categories":categories,
    },
    "force":{
        "initLayout": 'circular',
        "gravity": 0,
        "repulsion": 100,
        "edgeLength": 10
    }
})


## NER

In [94]:
!rm ../../pickles/swarog_data/graph_ner.sqlite

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
rm: cannot remove '../../pickles/swarog_data/graph_ner.sqlite': No such file or directory


In [96]:
import sqlite3
nergraph = sqlite3.connect('../../pickles/swarog_data/graph_ner.sqlite')

nergraph.execute("""
CREATE TABLE IF NOT EXISTS ner (
    body TEXT,
    label TEXT,
    docid NUMERIC
);
""")

nergraph.commit()

In [53]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")


In [98]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

nerka={}


for row in con.execute("select rowid,body,label from covid_fake_news_search"):
    docid, body, label = row
    ner_results = nlp(body)

    for ent in ner_results:
        #print(ent["entity"], ent['word'])
        nergraph.execute("""
                        insert into ner(docid,body, label) values(?,?,?)
                         """, (docid,json.dumps(ent,default=str),label))
        #if ent['word'] not in nerka:
        #    nerka[ent['word']] = []
        #nerka[ent['word']].append(docid)


In [116]:
nergraph.commit()
nergraph.close()

ProgrammingError: Cannot operate on a closed database.

In [100]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select * from ner limit 1

using ../../pickles/swarog_data/graph_ner.sqlite


,body,label,docid
0,"{""entity"": ""B-MISC"", ""score"": ""0.9705691"", ""in...",1,1


In [101]:
%%sql show=raw db=../../pickles/swarog_data/graph_ner.sqlite
ALTER TABLE ner add column word TEXT GENERATED ALWAYS AS (json_extract(body, '$.word')) VIRTUAL

using ../../pickles/swarog_data/graph_ner.sqlite


'(empty set)'

In [102]:
%%sql show=raw db=../../pickles/swarog_data/graph_ner.sqlite
ALTER TABLE ner add column entity TEXT GENERATED ALWAYS AS (json_extract(body, '$.entity')) VIRTUAL

using ../../pickles/swarog_data/graph_ner.sqlite


'(empty set)'

In [103]:
%%sql show=raw db=../../pickles/swarog_data/graph_ner.sqlite
create index ner_word on ner(word)

using ../../pickles/swarog_data/graph_ner.sqlite


'(empty set)'

### playground

In [108]:
%%sql show=df db=../../pickles/swarog_data/swarog.sqlite
select  label, count(*) from covid_fake_news_search 
group by 1 order by 2

using ../../pickles/swarog_data/swarog.sqlite


,label,count(*)
0,0,461
1,1,8511


In [107]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select  label, count(*) from ner 
group by 1 order by 2

using ../../pickles/swarog_data/graph_ner.sqlite


,label,count(*)
0,0,1545
1,1,28952


In [111]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select word, label, count(*) from ner 
where entity = "I-PER" and word not like "#%" and length(word) > 4
group by 1,2 order by 3 desc limit 10

using ../../pickles/swarog_data/graph_ner.sqlite


,word,label,count(*)
0,Gates,1,135
1,Trump,1,86
2,Obama,1,23
3,Francis,1,19
4,Ronald,1,19
5,Putin,1,17
6,Gandhi,1,15
7,FernÃ,1,11
8,Johnson,1,11
9,Raoul,1,10


In [113]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select  entity, count(*) from ner 
group by 1 order by 2 desc

using ../../pickles/swarog_data/graph_ner.sqlite


,entity,count(*)
0,B-MISC,5440
1,B-LOC,5342
2,I-PER,3876
3,I-ORG,3760
4,I-MISC,3593
5,I-LOC,2917
6,B-ORG,2802
7,B-PER,2767


## SVO

In [132]:
import sqlite3
nergraph = sqlite3.connect('../../pickles/swarog_data/graph_ner.sqlite')

nergraph.execute("""
CREATE TABLE IF NOT EXISTS svo (
    subject TEXT,
    object TEXT, 
    predicate TEXT, 
    label TEXT,
    docid NUMERIC
);
""")

nergraph.execute("""
delete from svo;
""")


nergraph.commit()

In [133]:
from posextract import grammatical_triples

for row in con.execute("select body, label, rowid from covid_fake_news_search order by 2"):
    triples = grammatical_triples.extract(row[0])
    for triple in triples:
        #print(triple)
        nergraph.execute(""" insert into svo(subject, predicate, object, label, docid) values (?,?,?,?,?) """, 
                         (triple.subject,triple.verb,triple.object,row[1],row[2]))
    # insertvalues = []
    # for triple in triples:
    #     insertvalues.append((triple.subject,triple.verb,triple.object,row[2],row[1]))
    # if len(insertvalues) > 0:
    #     con.executemany("""insert into triples(subjec, predicate, object, doc_rowid, label) values (?,?,?,?,?)""", insertvalues)

In [172]:
nergraph.commit()
nergraph.close()

ProgrammingError: Cannot operate on a closed database.

In [138]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select  subject, predicate, count(*) from svo 
group by 1,2 order by 3 desc limit 20

using ../../pickles/swarog_data/graph_ner.sqlite


,subject,predicate,count(*)
0,video,viewed,205
1,video,shows,161
2,Video,shows,148
3,photo,shared,100
4,photo,shows,97
5,coronavirus,is,92
6,COVID-19,is,90
7,it,shows,80
8,people,died,58
9,it,is,50


In [160]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select word, group_concat(distinct lower(predicate)), count(distinct ner.docid), count(distinct lower(predicate)) from ner 
join svo on (svo.docid = ner.docid)
where ner.entity in ("I-PER","B-PER")
and word not like "#%" and length(word)>4
group by 1
order by 3 desc
limit 20

using ../../pickles/swarog_data/graph_ner.sqlite


,word,group_concat(distinct lower(predicate)),count(distinct ner.docid),count(distinct lower(predicate))
0,Trump,"comes,is,leading,published,stopped,attributed,...",145,151
1,Gates,"filed,show,shows,is,planned,alter,change,said,...",128,127
2,Donald,"is,leading,published,stopped,caught,be,done,sa...",82,94
3,Obama,"visited,show,shows,mail,told,leave,is,speak,gi...",36,42
4,Anthony,"arrested,shows,delivered,said,are,claim,viewed...",32,40
5,Rodrigo,"questioned,criticizing,happened,is,knows,said,...",31,51
6,Putin,"arresting,died,spreading,stay,claims,follow,sh...",28,39
7,Ronald,"find,is,beaten up,earn,shows,bought,escape,go,...",25,26
8,Francis,"came,reports,tested,confirmed,allows,calling o...",22,24
9,Nancy,"include,tried,is,vote,said,accept,deleted,go,t...",22,27


## Edges 2

In [175]:
import sqlite3
nergraph = sqlite3.connect('../../pickles/swarog_data/graph_ner.sqlite')
proxgraph = sqlite3.connect('../../pickles/swarog_data/graph.sqlite')


In [177]:
nergraph.execute("""
CREATE TABLE IF NOT EXISTS prox (
    source     TEXT,
    target     TEXT,
    properties TEXT
);
""")

In [178]:
for row in proxgraph.execute("select * from edges "):
    nergraph.execute("insert into prox values(?,?,?)", row)

In [181]:
nergraph.commit()

In [187]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select 
    *
from prox
limit 2

using ../../pickles/swarog_data/graph_ner.sqlite


,source,target,properties
0,1,1,"{""rowid"": 1, ""length"": 206, ""hit_count"": 7, ""l..."
1,1,5,"{""rowid"": 5, ""length"": 144, ""hit_count"": 1, ""l..."


# Extract SVO Triples

In [ ]:
from posextract import grammatical_triples

In [220]:
%%sql show=raw
drop table if exists triples  

'(empty set)'

In [221]:
%%sql show=raw
create table triples(subjec, predicate, object, doc_rowid, label)

'(empty set)'

In [222]:
for row in con.execute("select body, label, rowid from covid_fake_news_search order by 2"):
    #print("->",row[0])
    triples = grammatical_triples.extract(row[0])
    insertvalues = []
    for triple in triples:
        insertvalues.append((triple.subject,triple.verb,triple.object,row[2],row[1]))
    if len(insertvalues) > 0:
        #print(insertvalues)
        con.executemany("""insert into triples(subjec, predicate, object, doc_rowid, label) values (?,?,?,?,?)""", insertvalues)

In [234]:
%%sql show=df
select predicate, count(*), 100-100.*sum(label)/count(*) 
from triples group by 1 order by 2 desc  limit 10

,predicate,count(*),100-100.*sum(label)/count(*)
0,is,954,2.515723
1,shows,750,1.600000
2,shared,443,0.225734
3,are,415,6.265060
4,says,326,5.521472
5,said,313,1.916933
6,claims,300,0.333333
7,died,228,5.701754
8,viewed,220,0.000000
9,be,193,5.699482


## checking 

In [296]:
%%sql show=df
select * from triples where predicate = "voted"

,subjec,predicate,object,doc_rowid,label
0,candidate,voted,leave,1576,0
1,parties,voted,proposal,586,1
2,Democrats,voted,stimulus package,6225,1
3,group,voted,bill,6349,1
4,Democrats,voted,stimulus bill,6571,1
5,Rodrigo Maia,voted,law,7032,1


In [290]:
raw[1575]

'Congressional candidate Chris Jacobs “voted against paid leave for COVID-19.”'

In [ ]:
%%sql show=raw
select * from rawsearch where body match "Chris+Jacobs"

In [295]:
resp,words=get_related_docs(raw[1575])
pd.DataFrame(resp)

common [(1576, 7), (1672, 2), (1845, 2), (7941, 2), (1731, 1), (6070, 1), (75, 1), (154, 1), (734, 1), (781, 1)]


,rowid,length,hit_count,label,dataset,hit_keywords,keywords,distance
0,1576,77,7,0,covid_fake_news,"[chris, jacobs, congressional, candidate, vote...","[chris, jacobs, congressional, candidate, vote...",0.000000
1,1672,203,2,0,covid_fake_news,"[paid, leave]","[estimated, benefits, receiving, package, busi...",0.714286
2,1845,253,2,1,covid_fake_news,"[paid, leave]","[march, 6th, workplaces, starting, paid, flori...",0.714286
3,7941,255,2,1,covid_fake_news,"[paid, leave]","[march, workplaces, starting, paid, florida, w...",0.714286
4,1731,220,1,1,covid_fake_news,[congressional],"[refugee, resettlement, waitress, congressiona...",0.857143
5,6070,234,1,1,covid_fake_news,[congressional],"[iâ, knowingâ, œthanks, refugee, resettlement,...",0.857143
6,75,92,1,1,covid_fake_news,[candidate],"[colon, cain, herman, republican, candidate, c...",0.857143
7,154,317,1,1,covid_fake_news,[candidate],"[authorised, upcoming, countryâ, candidate, ze...",0.857143
8,734,162,1,1,covid_fake_news,[candidate],"[guests, haddad, workersâ, birthday, celebrati...",0.857143
9,781,123,1,1,covid_fake_news,[candidate],"[advocating, guests, haddad, birthday, fernand...",0.857143


In [294]:
for ent in grammatical_triples.extract(raw[1575].split(".")):
    print(ent)

candidate voted against leave


In [59]:
example = 'WHO suggests - oral hygiene prevents coronavirus.'

for ent in grammatical_triples.extract(example.split(".")):
    print(ent)


hygiene prevents coronavirus
WHO suggests coronavirus


# SVO ver2

In [71]:
from subject_verb_object_extract import findSVOs, nlp
tokens = nlp("Seated in Mission Control, Chris Kraft neared the end of a tedious Friday afternoon as he monitored a seemingly interminable ground test of the Apollo 1 spacecraft.")
tokens = nlp("  WHO suggests oral hygiene to prevent the spread of coronavirus.")
svos = findSVOs(tokens)
print(svos)

[('WHO', 'suggests', 'oral hygiene'), ('oral hygiene', 'prevent', 'the spread of coronavirus')]


In [5]:
from subject_verb_object_extract import findSVOs, nlp
tokens = nlp("Seated in Mission Control, Chris Kraft neared the end of a tedious Friday afternoon as he monitored a seemingly interminable ground test of the Apollo 1 spacecraft.")
tokens = nlp("  WHO suggests oral hygiene to prevent the spread of coronavirus.")
svos = findSVOs(tokens)
print(svos)

def semantic_train(body, labels):
    

IndentationError: expected an indented block (1323189935.py, line 8)

In [6]:
import sqlite3
from tqdm import tqdm
import os
con=sqlite3.connect("svo.db")
con.execute("create table if not exists svo(sub, verb, obj, docid, label)")
con.commit()

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nernlp = pipeline("ner", model=model, tokenizer=tokenizer)
con.execute("create table if not exists ner(docid, js, label)")
con.commit()

In [8]:
import json
nerka={}
con.execute("delete from ner")
con.commit()
for docid, txt in tqdm(enumerate(body[train]), total=len(train)):
    ner_results = nernlp(txt)
    for ent in ner_results:
        con.execute("""
                    insert into ner(docid, js, label) values(?,?,?)
                     """, [docid,json.dumps(ent,default=str),int(labels[train][docid])])


NameError: name 'body' is not defined

In [9]:
con.commit()

In [10]:
from subject_verb_object_extract import findSVOs, nlp
con.execute("delete from svo")
con.commit()
for docid, txt in tqdm(enumerate(body[train]), total=len(train)):
    tokens = nlp(txt)
    svos = findSVOs(tokens)
    for svo in svos:
        if len(svo) == 3:
            con.execute("insert into svo values(?,?,?,?,?)", [svo[0],svo[1],svo[2],docid, int(labels[train][docid])] )


NameError: name 'body' is not defined

In [32]:
con.commit()

In [52]:
txt = body[train][13]
print(txt)
tokens = nlp(txt)
svos = findSVOs(tokens)
svos

There is evidence that chlorine dioxide cures COVID-19


[('There', 'is', 'evidence'), ('chlorine dioxide', 'cures', 'COVID-19')]

In [79]:
pd.read_sql("select obj, label, count(*) from svo where obj='COVID-19' group by 1,2 order by 3 desc", con)

,obj,label,count(*)
0,COVID-19,0,308
1,COVID-19,1,8


# Semantic relations

In [5]:
from subject_verb_object_extract import findSVOs, nlp
tokens = nlp("Seated in Mission Control, Chris Kraft neared the end of a tedious Friday afternoon as he monitored a seemingly interminable ground test of the Apollo 1 spacecraft.")
tokens = nlp("  WHO suggests oral hygiene to prevent the spread of coronavirus.")
svos = findSVOs(tokens)
print(svos)

def semantic_train(body, labels):
    

IndentationError: expected an indented block (1323189935.py, line 8)

In [6]:
import sqlite3
from tqdm import tqdm
import os
con=sqlite3.connect("svo.db")
con.execute("create table if not exists svo(sub, verb, obj, docid, label)")
con.commit()

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nernlp = pipeline("ner", model=model, tokenizer=tokenizer)
con.execute("create table if not exists ner(docid, js, label)")
con.commit()

In [8]:
import json
nerka={}
con.execute("delete from ner")
con.commit()
for docid, txt in tqdm(enumerate(body[train]), total=len(train)):
    ner_results = nernlp(txt)
    for ent in ner_results:
        con.execute("""
                    insert into ner(docid, js, label) values(?,?,?)
                     """, [docid,json.dumps(ent,default=str),int(labels[train][docid])])


NameError: name 'body' is not defined

In [9]:
con.commit()

In [10]:
from subject_verb_object_extract import findSVOs, nlp
con.execute("delete from svo")
con.commit()
for docid, txt in tqdm(enumerate(body[train]), total=len(train)):
    tokens = nlp(txt)
    svos = findSVOs(tokens)
    for svo in svos:
        if len(svo) == 3:
            con.execute("insert into svo values(?,?,?,?,?)", [svo[0],svo[1],svo[2],docid, int(labels[train][docid])] )


NameError: name 'body' is not defined

In [32]:
con.commit()

In [52]:
txt = body[train][13]
print(txt)
tokens = nlp(txt)
svos = findSVOs(tokens)
svos

There is evidence that chlorine dioxide cures COVID-19


[('There', 'is', 'evidence'), ('chlorine dioxide', 'cures', 'COVID-19')]

In [79]:
pd.read_sql("select obj, label, count(*) from svo where obj='COVID-19' group by 1,2 order by 3 desc", con)

,obj,label,count(*)
0,COVID-19,0,308
1,COVID-19,1,8


# Semantic relations

In [5]:
from subject_verb_object_extract import findSVOs, nlp
tokens = nlp("Seated in Mission Control, Chris Kraft neared the end of a tedious Friday afternoon as he monitored a seemingly interminable ground test of the Apollo 1 spacecraft.")
tokens = nlp("  WHO suggests oral hygiene to prevent the spread of coronavirus.")
svos = findSVOs(tokens)
print(svos)

def semantic_train(body, labels):
    

IndentationError: expected an indented block (1323189935.py, line 8)

In [6]:
import sqlite3
from tqdm import tqdm
import os
con=sqlite3.connect("svo.db")
con.execute("create table if not exists svo(sub, verb, obj, docid, label)")
con.commit()

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nernlp = pipeline("ner", model=model, tokenizer=tokenizer)
con.execute("create table if not exists ner(docid, js, label)")
con.commit()

In [8]:
import json
nerka={}
con.execute("delete from ner")
con.commit()
for docid, txt in tqdm(enumerate(body[train]), total=len(train)):
    ner_results = nernlp(txt)
    for ent in ner_results:
        con.execute("""
                    insert into ner(docid, js, label) values(?,?,?)
                     """, [docid,json.dumps(ent,default=str),int(labels[train][docid])])


NameError: name 'body' is not defined

In [9]:
con.commit()

In [10]:
from subject_verb_object_extract import findSVOs, nlp
con.execute("delete from svo")
con.commit()
for docid, txt in tqdm(enumerate(body[train]), total=len(train)):
    tokens = nlp(txt)
    svos = findSVOs(tokens)
    for svo in svos:
        if len(svo) == 3:
            con.execute("insert into svo values(?,?,?,?,?)", [svo[0],svo[1],svo[2],docid, int(labels[train][docid])] )


NameError: name 'body' is not defined

In [32]:
con.commit()

In [52]:
txt = body[train][13]
print(txt)
tokens = nlp(txt)
svos = findSVOs(tokens)
svos

There is evidence that chlorine dioxide cures COVID-19


[('There', 'is', 'evidence'), ('chlorine dioxide', 'cures', 'COVID-19')]

In [79]:
pd.read_sql("select obj, label, count(*) from svo where obj='COVID-19' group by 1,2 order by 3 desc", con)

,obj,label,count(*)
0,COVID-19,0,308
1,COVID-19,1,8


# Training

In [1]:
import numpy as np
import bentoml
from bentoml.io import NumpyNdarray
from bentoml.io import JSON
from annoy import AnnoyIndex
import re

import transformers
transformers.logging.set_verbosity_error()
from torch.utils.data import DataLoader 

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import TFDistilBertModel, DistilBertTokenizerFast
import torch
from scipy.sparse import csr_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
import pickle5 as pickle
import sqlite3

PATH_PICKLES = '../../pickles'
 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device:", device)

if "disilbert_model" not in locals():
    disilbert_tokenizer =  AutoTokenizer.from_pretrained("distilbert-base-uncased")
    disilbert_model = AutoModel.from_pretrained("distilbert-base-uncased")
    handle = disilbert_model.to(device)

class BERTEmbeddings:
    def __init__(self):
        self.tokenizer =  disilbert_tokenizer
        self.model = disilbert_tokenizer
        self.max_length = 256
        self.model_name = disilbert_model

    def fit(self, X=None, y=None):
        pass
    
    def encode(self, txt):
        return self.tokenizer(txt, max_length=self.max_length, 
                              truncation=True, padding=True, return_tensors="pt")

    def transform(self, X=None):
        dataloader = DataLoader(X, batch_size=4, shuffle=False)
        allembeds = []
        for batch in dataloader:
            batchenc = disilbert_tokenizer(batch, max_length=256, 
                                           truncation=True, padding=True, return_tensors="pt")
            input_ids = batchenc['input_ids'].to(device)
            attention_mask = batchenc['attention_mask'].to(device)
            batchout = disilbert_model(input_ids, attention_mask=attention_mask, 
                                       output_hidden_states=True)
            embeds = [vec[0].cpu().detach().numpy() for vec in batchout[1][-1]]
            allembeds.extend(embeds)
        return np.array(allembeds)

using device: cuda


In [6]:
%%sql show=df
select 
min(rowid-1) as low, 
max(rowid-1) as hi, 
count(*) as size,
sum(label) as fake,
-1*sum(label-1) as real,
dataset
from raw 
group by dataset
order by low

UsageError: Cell magic `%%sql` not found.


In [3]:
with open('../../pickles/swarog_data/swarog_bertcls.pickle', 'rb') as handle:
    dst = pickle.load(handle)
dst.shape

(186477, 768)

In [7]:
covid_fake_news_bert_cls = dst[0:8972]
labels = np.array([x[0] for x in con.execute("select label from raw where dataset='covid_fake_news'").fetchall()])
raw = np.array([x[0] for x in con.execute("select body from raw where dataset='covid_fake_news'").fetchall()])
covid_fake_news_bert_cls.shape, labels.shape

((8972, 768), (8972,))

In [8]:
from sklearn.model_selection import train_test_split
X=np.array(range(0,len(covid_fake_news_bert_cls)))
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.43, random_state=42)

In [10]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
from IPython.display import clear_output

from sklearn.linear_model import LogisticRegression

cls = LogisticRegression(max_iter=10000)
cls.fit(covid_fake_news_bert_cls[X_train], y_train)
y_pred = cls.predict(covid_fake_news_bert_cls[X_test])

In [11]:
balanced_accuracy_score(y_test,y_pred)

0.8088963404537175

In [196]:
[(x,) for x in X_train[:10]]

[(1224,),
 (142,),
 (4832,),
 (1490,),
 (7556,),
 (6306,),
 (1590,),
 (7504,),
 (4765,),
 (2653,)]

In [256]:
nergraph.execute("""
drop table trset
""")

In [257]:
nergraph.execute("""
create table IF NOT EXISTS trset(docid text)
""")

for v in X_train:
    nergraph.execute("""
    insert into trset(docid) values(?)
    """, [str(v+1)])

In [258]:
nergraph.commit()

In [259]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select * from svo where docid=1503


using ../../pickles/swarog_data/graph_ner.sqlite


,subject,object,predicate,label,docid
0,Gates Foundation,billions,spending,1,1503
1,Gates Foundation,number,spending,1,1503
2,Gates Foundation,now,spending,1,1503
3,people,COVID-19 vaccine,refuse,1,1503
4,it,microchips,include,1,1503
5,injections,chips,include,1,1503
6,procedures,chips,include,1,1503


In [252]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select id from trset where id = 1503

using ../../pickles/swarog_data/graph_ner.sqlite


,id
0,1503


In [255]:
1502 in X_train 

False

In [378]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select *, json_extract(properties,'$.hit_keywords') , json_extract(properties,'$.label')  from prox
where source = '1503'
and source <> target
and target in (select docid from trset)
limit 15

using ../../pickles/swarog_data/graph_ner.sqlite


,source,target,properties,"json_extract(properties,'$.hit_keywords')","json_extract(properties,'$.label')"
0,1503,2784,"{""rowid"": 2784, ""length"": 262, ""hit_count"": 7,...","[""include"",""procedures"",""dental"",""forthcoming""...",1
1,1503,527,"{""rowid"": 527, ""length"": 313, ""hit_count"": 1, ...","[""include""]",1
2,1503,2783,"{""rowid"": 2783, ""length"": 97, ""hit_count"": 1, ...","[""include""]",1
3,1503,2790,"{""rowid"": 2790, ""length"": 100, ""hit_count"": 1,...","[""include""]",1


In [370]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select * from svo
where  docid = '1443'
limit 5

using ../../pickles/swarog_data/graph_ner.sqlite


,subject,object,predicate,label,docid
0,Joe Biden,Americans,prosecute,1,1443


In [371]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select * from ner
where  docid = '1443'
limit 10

using ../../pickles/swarog_data/graph_ner.sqlite


,body,label,docid,word,entity
0,"{""entity"": ""B-PER"", ""score"": ""0.99967194"", ""in...",1,1443,Joe,B-PER
1,"{""entity"": ""I-PER"", ""score"": ""0.99967575"", ""in...",1,1443,B,I-PER
2,"{""entity"": ""B-PER"", ""score"": ""0.50980306"", ""in...",1,1443,##iden,B-PER
3,"{""entity"": ""B-MISC"", ""score"": ""0.9997353"", ""in...",1,1443,Democrats,B-MISC
4,"{""entity"": ""B-MISC"", ""score"": ""0.99956316"", ""i...",1,1443,Americans,B-MISC


In [358]:
%%sql show=df db=../../pickles/swarog_data/graph_ner.sqlite
select * from ner
where  word = 'Jacobs'
limit 10

using ../../pickles/swarog_data/graph_ner.sqlite


,body,label,docid,word,entity
0,"{""entity"": ""I-PER"", ""score"": ""0.99975437"", ""in...",0,1576,Jacobs,I-PER


In [334]:
nergraph.close()

In [16]:
def rule1(con,docid):
    data = con.execute(f"""
    select *, json_extract(properties,'$.hit_count') as ht  from prox
    where source = {docid}
    and source <> target
    and json_extract(properties,'$.label') = 0
    and target in (select docid from trset)
    and ht >= 5
    limit 15
    """).fetchall()
    
    r = len(data)
    #print(data,"d=",docid, "r=",r)
    return r

def rule2(con,docid):
    data = con.execute(f"""
    select *, json_extract(properties,'$.hit_count') as ht  from prox
    where source = {docid}
    and source <> target
    and json_extract(properties,'$.label') = 1
    and target in (select docid from trset)
    and ht > 5
    limit 15
    """).fetchall()
    
    r = len(data)
    #print(data,"d=",docid, "r=",r)
    return r

In [349]:
rule1(nergraph,'1576')

1

In [300]:
1 if rule1(6204) > 0 else 0

1

## errors

In [12]:
new_pred = []
new_pred.extend(y_pred)

In [13]:
balanced_accuracy_score(y_test,new_pred)

0.8088963404537175

In [14]:
nergraph = sqlite3.connect('../../pickles/swarog_data/graph_ner.sqlite')

In [17]:

for i in range(len(y_pred)):
    if y_pred[i] < 1:
        docid = X_test[i]+1
        rr = rule2(nergraph,docid)
        new_pred[i] = 1 if rr > 0 else 0
        #print(i, docid, y_pred[i], new_pred[i], rr)
        


In [18]:
for i in range(len(y_pred)):
    if y_pred[i] > 0:
        docid = X_test[i]+1
        rr = rule1(nergraph,docid)
        new_pred[i] = 0 if rr > 0 else 1
        #print(i, docid, y_pred[i], new_pred[i], rr)
        


In [ ]:
nergraph.close()

In [409]:
balanced_accuracy_score(y_test,new_pred)

0.809716012584865

In [19]:
balanced_accuracy_score(y_test,new_pred)

0.826982944196059

In [376]:
for i in range(len(y_pred)):
    if y_pred[i] < y_test[i]:
        print(i, y_pred[i], y_test[i], X_test[i]+1, raw[X_test[i]])

11 0 1 1503 “Due to the large number of people who will refuse the forthcoming COVID-19 vaccine because it will include tracking microchips, the Gates Foundation is now spending billions to ensure that all medical and dental injections and procedures include the chips.”
262 0 1 1296 WebQoof Recap: TISS Prof Falsely Targeted; Ramdev’s COVID-19 Cure
352 0 1 1531 $100B contact tracing bill “is about controlling/tracking population, not about coronavirus.”
522 0 1 1594 “Trump and McConnell are blocking stimulus checks for Americans married to immigrants.”
616 0 1 1784 Says the New York Times exposed the “real reason behind coronavirus hype: Crash the market to harm Trump’s re-election chances.”
660 0 1 1394 The Trump administration has had “zero unfulfilled equipment and supply requests from state governors.”
755 0 1 1769 “Go to 2020census.gov and fill out this census form so you can get your stimulus check.”
799 0 1 7208 “U.S. deaths since 1/22/20: COVID-19, 27; flu, 4,700; abortion, 118,

In [302]:
%%sql show=df
select label, count(*) from raw where dataset="pubhealth" group by 1

,label,count(*)
0,0,3769
1,1,6306


# Graph Emb

In [171]:
X_train

array([1224,  142, 4832, ..., 5390,  860, 7270])